In [18]:
import pandas as pd
import sys
import json

In [10]:
df = pd.read_csv('../Data/transformed_2017.csv', nrows = 1000)

In [19]:
def read_data():
    data = []
    f = '../Data/codebook_21.txt'
    with open(f) as f_pointer:
        for row in f_pointer.readlines():
            data.append(row.strip('/n'))
    return data

In [22]:
def is_int(row, meta_heading):
#     if meta_heading == 'FARM':
#         print (row)

    try:
        int(row[0])
        return True
    except:
        return False
      
def is_metaheading(row):
    if '\t' in row and row[0].isalpha():
        return True
    else:
        return False
    
def is_subheader(row, meta_heading):
    
    # these are meta headings where the keys have tabs
    # this throws off the logic below, but b/c we know from the codebook that they have no subheadings
    # we automatically return False
    if meta_heading in ['FARM','UNITSSTR', 'RELATE', 'RELATED', 'WKSWORK2', 'MULTGEND', 'METRO',
                        'WORKEDYR', 'SCHLTYPE', 'VETVIETN', 'EMPSTATD', 'MIGRATE1D', 'LANGUAGE',
                       'GQ', 'MIGPLAC1']:
        return False

    a = row[0].isalpha()
    b = '\t' not in row
    c = ('-' in row) or (':' in row)
    
    rt = (a and b) or c

    
    return rt

def is_header_break(row):
    return row == '\n'

# This skips subheadings right now by a list
# But are there any subheadings that we want stored in the old way right now? Unknown. 
def create_codebook_store(data):
    store = {}
    
    sub_headings_direct = ['REGION', 'GQ', 'ACREHOUS', 'BPL']
    has_subheading = False
    sub_heading_direct_placement = False
    
    for idx, row in enumerate(data):
        
        # find the meta heading and create that dictionary 
        if is_metaheading(row):
            meta_heading = row.split('\t') [0]
            meta_dict = {}
            has_subheading = False

            if meta_heading in sub_headings_direct:
                sub_heading_direct_placement = True
            continue


        # find the sub headings and create inner dictionaries
        elif is_subheader(row, meta_heading):

            sub_heading = row.strip('\n')
            
            has_subheading = True

            if not sub_heading_direct_placement:
                meta_dict[sub_heading] = {}
                
            continue

        # find the integers and add them, along with their meanings, to the inner dictionary
        elif is_int(row, meta_heading):
            
            lines = row.split('\t')
            key = int(lines[0].strip('/t'))
            value = lines[-1].strip('\n')
            
            if has_subheading and not sub_heading_direct_placement:
                meta_dict[sub_heading][key] = value
            
            elif has_subheading and sub_heading_direct_placement:
                meta_dict[key] = sub_heading
            
            else:
                meta_dict[key] = value
            
            continue

        # find the header break
        elif is_header_break(row):
            store[meta_heading] = meta_dict
            has_subheading = False
            skip_subheading = False
 
    return store 

data = read_data()
store = create_codebook_store(data)

## First, we need to handle the columns with missing values. 
Many of the columns indicate that a missing value can also mean no. Let's see which columns have missing values

In [15]:
missing = []

for h in list(df):
    if df[h].isnull().any():
        missing.append(h)

In [16]:
len(missing)

34

In [17]:
len(list(df))

110

## Next, let's create a mapper
That can map a column to how it represents missing values

In [23]:
data = read_data()
store = create_codebook_store(data)

In [25]:
missing_value_indicators = ['not identified', 'or N/A', 'N/A']

In [26]:
numerical_columns = ['INCOME', 'HHWT', 'FARMPROD', 'MORTAMT1', 'PROPINSR', 'RENTGRS', 'CONDOFEE', 'MOBLHOME'
                ,'COSTELEC', 'COSTGAS', 'COSTWATR', 'COSTFUEL', 'HHINCOME', 'VALUEH', 'PERWT',
                'MOMLOC', 'POPLOC', 'BIRTHYR', 'RACED', 'YRIMMIG', 'YRSUSA1', 'OCC', 'IND', 'INCTOT',
                'FTOTINC', 'INCSS', 'INCWELFR', 'INCRETIR', 'POVERTY', 'OCCSCORE', 'SEI', 'HWSEI',
                'PRESGL', 'PRENT', 'MOVEDIN', 'RIDERS', 'TRANTIME', 'DEPARTS', 'ROOMS']

For the numerical columns, we are going to assume that if all or all-minus-one digits are 9's, that is a non-relevant indicator. Similar with 00's. 